In [211]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = -1
print('Done')

Done


In [212]:
bechdel_df = pd.read_csv('bechdel.csv',encoding='utf-8')
bechdel_df = bechdel_df.drop('imdbid',axis=1)
bechdel_df = bechdel_df.set_index('imdb_id')

bechdel_df['soft_result'] = bechdel_df['rating'].replace({0: 'Fewer than two women', 1: 'Women don\'t talk to each other', 2: 'Women talk about a man', 3: 'Passes Bechdel test'})
bechdel_df['hard_result'] = bechdel_df['rating'].replace({0: 'Fail', 1: 'Fail', 2: 'Fail', 3: 'Pass'})

bechdel_df=bechdel_df.rename(columns = {'id': 'bechdel_site_id', 'year': 'bechdel_year','title':'bechdel_title'})

bechdel_df.shape # 7,253 rows x 10 columns
#print('Done')
#bechdel_df[6900:6910]

(7253, 10)

In [213]:
omdb_df = pd.read_csv('omdb.csv',encoding='utf-8')
omdb_df['imdb_link'] = 'http://www.imdb.com/title/'+omdb_df['imdb_id_str']
omdb_df = omdb_df.drop('imdb_id_str',axis=1)
omdb_df['imdb_id']  = pd.to_numeric(omdb_df['imdb_id'])
omdb_df = omdb_df.set_index('imdb_id')
#omdb_df
omdb_df = omdb_df.drop('type',axis=1)
#print(len(omdb_df.columns))
#print(omdb_df.columns)
cols = ['imdbtitle','imdbyear','studio','nVotes','userscore','metascore']
cols += ['imdb_release_date','mpaa_rating','imdb_boxoffice']
cols += ['imdb_link','english','usa','imdb_link','genres']
cols += ['runtime','imdb_plot','awards','director','writers','actors','posterlink','website']
cols += ['dvd_release_date','languages','countries','all_ratings_list','response','Error']
omdb_df = omdb_df[cols]
#print(omdb_df.columns)
#omdb_df.loc[3].transpose()
omdb_df.shape  # 7,156 rows x 28 columns
#omdb_df.columns
#print('Done')

(7156, 28)

In [214]:
#bechdel_df.groupby(pd.isnull(bechdel_df['bechdel_site_id'])).size()  # False: 7253 (all rows have bechdel_site_id)
#omdb_df.groupby(pd.isnull(omdb_df['imdbyear'])).size() # False: 7155 (all rows have )

joined_df = bechdel_df.join(omdb_df,how='outer')

#joined_df.groupby(pd.isnull(joined_df['imdbyear'])).size() # 7155 rows have OMDb data also

# All matches probably correct because we used imdb_id to match... 
### But let's investigate a 1% random sample just to make sure
#bechdel_omdb_join_test_df = joined_df[['bechdel_title','bechdel_year','imdbtitle','imdbyear']].sample(frac=0.01)
#bechdel_omdb_join_test_df.to_csv('bechdel_omdb_join_test.csv',encoding='utf-8')
#print('Matcher test file written as bechdel_omdb_join_test.csv.')
# Match looks good to Jordan! 2017-06-25 16:48:00
#joined_df.shape # 7,253 rows x 37 colummns
#print('Done')

joined_df.shape
#joined_df[['bechdel_title','bechdel_year','imdbtitle','imdbyear']][joined_df['bechdel_title'] != joined_df['imdbtitle']]

(7253, 38)

In [215]:

llaves_df = pd.read_csv('llaves.csv',encoding='utf-8')
#llaves_df[llaves_df['llave'].str.contains('alleg')]
#print(llaves_df[llaves_df['llave'] == 'allegia16iant'])
#print('\n')
#print(joined_df[['imdbtitle','imdbyear']][joined_df['imdbtitle'] == 'Allegiant'])
#joined_df.index.is_unique  # True
#llaves_df.index.is_unique  # True
#joined_df[joined_df.]

joined_df_con_llaves = pd.merge(joined_df, llaves_df, how='outer', on=['imdbtitle','imdbyear'],right_index=True)

#print(joined_df_con_llaves[['llave','imdbtitle','imdbyear']][joined_df_con_llaves['imdbtitle'] == 'Allegiant'])
print(joined_df_con_llaves.shape)

(13143, 39)


In [216]:

#joined_df[['imdbtitle','imdbyear']].head(1)
#llaves_df.head(40)
joined_df_con_llaves = pd.merge(joined_df, llaves_df, how='outer', on=['imdbtitle','imdbyear'],right_index=True)

#joined_df[['imdbtitle','imdbyear','llave']][pd.isnull(joined_df_con_llaves['llave']) == False].sort_values('llave')

joined_df_con_llaves['noa'] =  np.where((joined_df_con_llaves['imdbtitle'].str.slice(0,2) == 'A '),(joined_df_con_llaves['imdbtitle'].str.slice(2,len(joined_df_con_llaves['imdbtitle']))),joined_df_con_llaves['imdbtitle'])

joined_df_con_llaves['nothe'] =  np.where((joined_df_con_llaves['noa'].str.slice(0,4) == 'The '),(joined_df_con_llaves['noa'].str.slice(4,len(joined_df_con_llaves['noa']))),joined_df_con_llaves['noa'])

joined_df_con_llaves['fixedtitle'] = joined_df_con_llaves['nothe'].str.lower().str.replace(' ','')

joined_df_con_llaves['autollave'] = joined_df_con_llaves['fixedtitle'].str.slice(0,7)
joined_df_con_llaves['autollave'] = joined_df_con_llaves['autollave'].str.cat(joined_df_con_llaves['imdbyear'].astype('str').str.slice(2,4),sep='')
joined_df_con_llaves['autollave'] = joined_df_con_llaves['autollave'].str.cat(joined_df_con_llaves['fixedtitle'].astype('str').str.lower().str.replace(' ','').str.slice(-4,len(joined_df_con_llaves['fixedtitle'].astype('str'))),sep='')
joined_df_con_llaves['autollave'] = joined_df_con_llaves['autollave'].str.replace('(','[').str.replace(')',']')

joined_df_con_llaves['llave'] =  np.where((pd.isnull(joined_df_con_llaves['llave'])),(joined_df_con_llaves['autollave']),joined_df_con_llaves['llave'])
joined_df_con_llaves = joined_df_con_llaves.drop(['noa','nothe','fixedtitle','autollave'],axis=1)

#joined_df_con_llaves[['imdbtitle','imdbyear','llave','autollave']][joined_df_con_llaves['imdbtitle']=='Æon Flux']#.dropna().str.contains('Abandoned')]

# Remove any rows without Bechdel ratings
joined_df_con_llaves = joined_df_con_llaves[pd.isnull(joined_df_con_llaves['soft_result']) == False]
#joined_df_con_llaves[['imdbtitle','imdbyear','llave','userscore','metascore']][~pd.isnull(joined_df_con_llaves['metascore'])]

#print('Done!')
joined_df_con_llaves.shape # 7,253 rows x 39 columns

(7253, 39)

In [ ]:
boxofficemojo_df = pd.read_csv('boxofficemojo2000s_out.csv',encoding='utf-8')
boxofficemojo_df = boxofficemojo_df.set_index('key')
boxofficemojo_df

In [311]:
from datetime import datetime
revenue_df = pd.read_csv('revenue.csv',encoding='utf-8')

revenue_df = revenue_df.replace([u'',u'nan'],[np.nan,np.nan])
revenue_df['revenue'] = revenue_df['revenue'].str.replace('$','').str.replace(',','')
revenue_df['revenue'] = pd.to_numeric(revenue_df['revenue'])

revenue_df['budget'] = revenue_df['budget'].str.replace('$','').str.replace(',','')
revenue_df['budget'] = pd.to_numeric(revenue_df['budget'])

revenue_df['the_numbers_url'] = 'http://www.the-numbers.com/movie/' + revenue_df['title'].str.replace(' ','-')

revenue_df['noa'] =  np.where((revenue_df['title'].str.slice(0,2) == 'A '),(revenue_df['title'].str.slice(2,len(revenue_df['title']))),revenue_df['title'])

revenue_df['nothe'] =  np.where((revenue_df['noa'].str.slice(0,4) == 'The '),(revenue_df['noa'].str.slice(4,len(revenue_df['noa']))),revenue_df['noa'])

revenue_df[revenue_df['nothe'] == '12 Disasters of Christmas']

revenue_df['fixedtitle'] = revenue_df['nothe'].str.lower().str.replace(' ','')

revenue_df['fixedtitle'] = revenue_df['fixedtitle'].str.replace('(','[').str.replace(')',']')

#revenue_df['noa_end'] = np.where((revenue_df['fixedtitle'].str.lower().str.slice(-3,len(revenue_df['fixedtitle'])) == ', a'),'XXXXXXXX','y')

#revenue_df['nothe_end'] = np.where((revenue_df['fixedtitle'].str.lower().str.slice(-5,len(revenue_df['fixedtitle'])) == ', the'),'XXXXXXXX','y')

#np.where((revenue_df['title'].str.slice(0,2) == 'A '),(revenue_df['title'].str.slice(2,len(revenue_df['title']))),revenue_df['title'])

revenue_df['fixedtitle'] = np.where((revenue_df['fixedtitle'].str.slice(-4,len(revenue_df['fixedtitle'])) == ',a'),(revenue_df['fixedtitle'].str.slice(0,-2)),(revenue_df['fixedtitle']))

revenue_df['fixedtitle'] = np.where((revenue_df['fixedtitle'].str.slice(-4,len(revenue_df['fixedtitle'])) == ',the'),(revenue_df['fixedtitle'].str.slice(0,-4)),(revenue_df['fixedtitle']))

revenue_df['llave'] = revenue_df['fixedtitle'].str.slice(0,7)
revenue_df['llave'] = revenue_df['llave'].str.cat(revenue_df['year'].astype('str').str.slice(2,4),sep='')
revenue_df['llave'] = revenue_df['llave'].str.cat(revenue_df['fixedtitle'].str.slice(-4,len(revenue_df['fixedtitle'])))

#revenue_df.loc[1425]
revenue_df = revenue_df.drop(['noa','nothe','fixedtitle'],axis=1)

revenue_df = revenue_df.drop('id',axis=1)

revenue_df = revenue_df.set_index('llave')

print('Start with revenue data for {0:,.0f} films.'.format(len(revenue_df.index)))
# Keep only movies released this year or earlier
revenue_df = revenue_df[revenue_df['year'] <= datetime.now().year]
print('Keep 2017 or earlier movies. Now have revenue data for {0:,.0f} films.'.format(len(revenue_df.index)))

# Delete films with 'Untitled' in the name... except for the real 2009 film called '(Untitled)'
#revenue_df = revenue_df[(revenue_df['title'].str.contains('Untitled') == True) & (revenue_df['title'].str.contains('(Untitled)') == False)]
untitled_films = revenue_df[(revenue_df['title'].str.contains('Untitled') == True)]
untitled_films = untitled_films[untitled_films['year'] != 2009]
revenue_df = revenue_df[~revenue_df['title'].isin(untitled_films['title'].tolist())]
print('Keep movies with for-reals titles. Now have revenue data for {0:,.0f} films.'.format(len(revenue_df.index)))

#revenue_df
llaves_duplicados = revenue_df.index.get_duplicates() # first try, there are 52 duplicate key values. Let's see them.
#print(revenue_df[revenue_df.index.isin(llaves_duplicados)])

# Set them one-by-one, using key as we constructed it in prior research.

print('Before: {0} duplicate keys found...'.format(len(revenue_df.index.get_duplicates()))) # first try, there are 52 duplicate values. Let's see them.

revenue_df.index = np.where(((revenue_df.index == 'abandon16oned') & (revenue_df['title'] == 'Abandoned')), 'abandon15edvi', revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'abandon16oned') & (revenue_df['title'] == 'The Abandoned')), 'confine15ines', revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'afterma14math') & (revenue_df['genre'] == 'Horror')), 'afterma15hvii',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'afterma14math') & (revenue_df['genre'] == 'Black Comedy')), 'afterma13athi',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'america12ream') & (revenue_df['title'] == 'The American Dream')), 'america12drem',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'berline17cert') & (revenue_df['title'] == 'Berliner Philharmoniker Live 2016 - New Year\'s Gala Concert')), 'berline17nwyr',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'berline17cert') & (revenue_df['title'] == 'Berliner Philharmoniker Live: June Concert')), 'berline17june',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'branagh16tre]') & (revenue_df['title'] == 'Branagh Theatre Live: Romeo & Juliet 2016 (Theatre)')), 'branagh16romo',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'branagh16tre]') & (revenue_df['title'] == 'Branagh Theatre Live: The Entertainer 2016 (Theatre)')), 'branagh16entr',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'captive15tive') & (revenue_df['year'] == 1915)), 'captive151915',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'carebea10-lot') & (revenue_df['title'] == 'Care Bears: Adventures in Care-a-lot')), 'carebea10care',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'carebea10-lot') & (revenue_df['title'] == 'Care Bears: Share-a-Lot in Care-a-Lot')), 'carebea10shar',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'confirm16tion') & (revenue_df['title'] == 'Confirmation')), 'confirm16nthe',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'darkhor16orse') & (revenue_df['title'] == 'The Dark Horse')), 'darkhor14orse',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'deal08deal') & (revenue_df['title'] == 'Deal')), 'deali08eali',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'deal08deal') & (revenue_df['title'] == 'The Deal')), 'deali08thei',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'demente12nted') & (revenue_df['title'] == 'The Demented')), 'demente13nted',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'diaryof15girl') & (revenue_df['title'] == 'Diary of a Lost Girl')), 'diaryof15lstg',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'escaper16room') & pd.isnull(revenue_df['genre'])), 'escaper17rooi',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'escaper16room') & (revenue_df['genre'] == 'Thriller/Suspense')), 'escaper17roii',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'finders15pers') & (revenue_df['genre'] == 'Documentary')), 'finders15ersi',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'globeon16tre]') & (revenue_df['title'] == 'Globe On Screen: Measure For Measure - Shakespeare\'s Globe 2015 (Theatre)')), 'globeon16meas',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'globeon16tre]') & (revenue_df['title'] == 'Globe on Screen: Richard II - Shakespeare\'s Globe 2015 (Theatre)')), 'globeon16rcii',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'hideand16seek') & (revenue_df['title'] == 'Hide and Seek')), 'zhuomic16cang',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'hideand16seek') & (revenue_df['title'] == 'Hide and Never Seek')), 'hideand16nvsk',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'home09home') & (revenue_df['releasedate'] == '2009-04-23')), 'homei08omei',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'home09home') & (revenue_df['releasedate'] == '2009-11-27')), 'homeii08meii',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'home09home') & (revenue_df['releasedate'] == '2009-06-05')), 'homei09omei',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'hotel03otel') & (revenue_df['releasedate'] == '2003-07-25')), 'hotel01otel',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'indian09dian') & (revenue_df['releasedate'] == '2009-11-24')), 'indian07dian',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'indian09dian') & (revenue_df['releasedate'] == '2009-12-31')), 'deindia09iaan',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'killers15lers') & (revenue_df['title'] == 'Killers')), 'killers14lers',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'killers15lers') & (revenue_df['title'] == 'The Killers')), 'killers46lers',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'kizumon16-hen') & (revenue_df['title'] == 'Kizumonogatari II: Nekketsu-hen')), 'kizumon16-nek',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'kizumon16-hen') & (revenue_df['title'] == 'Kizumonogatari III: Reiketsu-hen')), 'kizumon16-rek',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'lamb16lamb') & (revenue_df['releasedate'] == '2016-01-08')), 'lambi15ambi',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'lamb16lamb') & (revenue_df['releasedate'] == '2016-09-06')), 'lambii15mbii',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'littles16mber') & (revenue_df['title'] == 'LittleScreen November')), 'littles16mnov',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'littles16mber') & (revenue_df['title'] == 'LittleScreen December')), 'littles16mdec',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'looneyt03tion') & (revenue_df['title'] == 'Looney Tunes: Stranger Than Fiction')), 'looneyt03sfic',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'metoper16016]') & (revenue_df['title'] == 'Met Opera Don Giovanni (2016)')), 'metoper16anni',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'metoper16016]') & (revenue_df['title'] == 'Met Opera: La Boheme (2016)')), 'metoper16heme',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'momentu15ntum') & (revenue_df['genre'] == 'Thriller/Suspense')), 'momentu15tumi',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'penthou10ouse') & (revenue_df['title'] == 'Penthouse')), 'penthou33ouse',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'residen12tion') & (revenue_df['title'] == 'Resident Evil Damnation')), 'resident12damn',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'return07turn') & (revenue_df['title'] == 'The Return')), 'returni06rnii',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'strawbe10ovie') & (revenue_df['title'] == 'Strawberry Shortcake: The Berryfest Princess Movie')), 'strawbe10berr',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'strawbe10ovie') & (revenue_df['title'] == 'Strawberry Shortcake: The Glimmerberry Ball Movie')), 'strawbe10glim',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'strawbe11eams') & (revenue_df['title'] == 'Strawberry Shortcake: Bright Lights Big Dreams')), 'strawbe11bbig',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'strawbe11eams') & (revenue_df['title'] == 'Strawberry Shortcake: Growing Up Dreams')), 'strawbe11grow',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'survivo15ivor') & (revenue_df['releasedate'] == '2015-05-29')), 'survivo15vori',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'survivo15ivor') & (revenue_df['releasedate'] == '2015-02-17')), 'survivo14vori',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'tempest12pest') & (revenue_df['releasedate'] == '2012-05-11')), 'tempest10stiv',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'tempest12pest') & (revenue_df['releasedate'] == '2012-08-07')), 'tempest79pest',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'thirst09irst') & (revenue_df['releasedate'] == '2009-07-31')), 'bakjwi09kjwi',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'woman11oman') & (revenue_df['title'] == 'A Woman')), 'woman10oman',revenue_df.index)
revenue_df.index = np.where(((revenue_df.index == 'woman11oman') & (revenue_df['title'] == 'The Woman')), 'womani11mani',revenue_df.index)

revenue_df = revenue_df[~((revenue_df.index == 'afterma17math') & (revenue_df['releasedate'] == '2017-12-31'))] # Future movie, delete
revenue_df = revenue_df[~((revenue_df.index == 'alicein10land') & (revenue_df['releasedate'] == '2010-03-02'))] # Was a re-release, delete
revenue_df = revenue_df[~((revenue_df.index == 'clashof81tans') & (revenue_df['releasedate'] == '1981-12-31'))] # Looks like a repeat
revenue_df = revenue_df[~((revenue_df.index == 'closert15moon') & (revenue_df['releasedate'] == '2015-04-17'))] # Remove earlier limited release
revenue_df = revenue_df[~((revenue_df.index == 'closet01oset') & (revenue_df['releasedate'] == '2001-04-06'))] # Remove earlier limited release
revenue_df = revenue_df[~((revenue_df.index == 'dog\'sbr07fast') & (revenue_df['genre'] == 'Comedy'))] # Duplicate entry
revenue_df = revenue_df[~((revenue_df.index == 'fever16ever') & (revenue_df['releasedate'] == '2016-12-06'))] # Remove earlier limited release
revenue_df = revenue_df[~(revenue_df.index == 'it17it')] # Both are future movies, delete
revenue_df = revenue_df[~((revenue_df.index == 'killswi17itch') & (revenue_df['title'] == 'Kill Switch'))] # Future movie, delete
revenue_df = revenue_df[~((revenue_df.index == 'landbef16rave') & (revenue_df['title'] == 'Land Before Time: Journey of the Brave'))] # Duplicate, delete row with less information
revenue_df = revenue_df[~(revenue_df.index == 'metoper17ette')] # Both are future movies, delete
revenue_df = revenue_df[~((revenue_df.index == 'nightli14ghts') & (revenue_df['title'] == 'Nightlights'))] # Duplicate
revenue_df = revenue_df[~((revenue_df.index == 'papercl04lips') & (revenue_df['title'] == 'Paper Clips'))] # Duplicate
revenue_df = revenue_df[~((revenue_df.index == 'ritual13tual') & (pd.isnull(revenue_df['genre'])))] # Duplicate entry
revenue_df = revenue_df[~((revenue_df.index == 'secretl13itty') & (revenue_df['releasedate'] == '2013-12-10'))] # Remove earlier limited release
revenue_df = revenue_df[~((revenue_df.index == 'sleepin11auty') & (revenue_df['title'] == 'The Sleeping Beauty'))] # One-time performance
revenue_df = revenue_df[~(revenue_df.index == 'souveni17enir')] # Both are future movies, delete
revenue_df = revenue_df[~((revenue_df.index == 'thirst09irst') & (revenue_df['releasedate'] == '2009-12-31'))] # Duplicate
revenue_df = revenue_df[~((revenue_df.index == 'strawdo11dogs') & (pd.isnull(revenue_df['genre'])))] # Duplicate entry

print('After: {0} duplicate keys found!'.format(len(revenue_df.index.get_duplicates())))
#print('Done!')
llaves_duplicados = revenue_df.index.get_duplicates() # second try, there are hopefully no duplicate key values. 
#revenue_df[revenue_df.index.isin(llaves_duplicados)].sort_index()


# MATCHING UP ON LLAVES AS A RESULT OF THE MATCHING WORK BELOW. CHANGE THE LLAVE HERE TO THE LLAVE FROM BECHDEL/OMBD DATA.
revenue_df.index = np.where((revenue_df.index == 'intoler16ance'), 'intoler16ages',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'bronyen25mkin'), 'battles25mkin',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'casabla42anca'), 'casabla43anca',revenue_df.index)  
revenue_df.index = np.where((revenue_df.index == 'aboutti13time'), 'aboutti13imei',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'adam09adam'), 'adami09dami',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'adam&st06teve'), 'adam&st05teve',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'adam\'sa07ples'), 'adam\'sa05ples',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'adam\'sa07ples'), 'adam\'sa05ples',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'adaptat02tion'), 'adaptat02ion.',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'adultwo14orld'), 'adultwo13orld', revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'adventu05n3-d'), 'adventu05l3-d',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'fondkis04kiss'), 'aefondk04s...',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'after.l10life'), 'after.l09life',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'agora10gora'), 'agora09gora',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'ahivael13ablo'), 'ahivael12ablo',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'aimée&j00guar'), 'aimee&j99guar',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'ajami10jami'), 'ajami09jami',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'alicedo75more'), 'alicedo74more',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'aloft15loft'), 'aloft14loft',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'alpeis12peis'), 'alps11alps',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'amandla03mony'), 'amandla02mony',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'amateur95teur'), 'amateur94teur',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'lefabul01lain'), 'amélie01élie',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'amen03amen'), 'amen.02men.',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'america09olet'), 'america08olet',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'america08mbie'), 'america07mbie',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'amoresp01rros'), 'amoresp00rros',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'amourfo15rfou'), 'amourfo14rfou',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'aninvis11sign'), 'aninvis10sign',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'angel-a07el-a'), 'angel-a05el-a',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'angryin16dess'), 'angryin15dess',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'nationa78ouse'), 'animalh78ouse',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'annakar12nina'), 'annakar12inai',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'antonia96onia'), 'antonia95line',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'emergo12ergo'), 'apartme11t143',revenue_df.index)  
revenue_df.index = np.where((revenue_df.index == 'appurus05hîdo'), 'applese04seed',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'appropr15vior'), 'appropr14iour',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'aprèsvo05s...'), 'aprèsvo03vous',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'armyofd93ness'), 'armyofd92ness',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'karigur12etti'), 'secretw10etty',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'astérix99ésar'), 'asterix99esar',revenue_df.index)  
revenue_df.index = np.where((revenue_df.index == 'atmiddl14eton'), 'atmiddl13eton',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'atlassh11art1'), 'atlassh11arti',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'attenbe12berg'), 'attenbe12berg',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'ôdishon01shon'), 'auditio99tion',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'aufdera08eite'), 'edgeofh07aven',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'augusti13tine'), 'augusti12tine',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'elaura06aura'), 'aura05aura',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'fauteil07stre'), 'avenuem06igne',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'avrilet16uqué'), 'aprilan15orld',revenue_df.index)
revenue_df.index = np.where((revenue_df.index == 'awakeni12ning'), 'awakeni11ingi',revenue_df.index)

revenue_df.index.name = 'llave'
revenue_df = revenue_df.reset_index()

revenue_df = revenue_df[revenue_df['revenue'] > 0]
print('Keep movies with box office releases, meaning revenue > 0. Now have revenue data for {0:,.0f} films.'.format(len(revenue_df.index)))

all_joined_df = pd.merge(joined_df_con_llaves,revenue_df,how='outer',on='llave',right_index=True)

#print('Done!')

#print(joined_df_con_llaves.shape)
#print(joined_df_con_llaves[['llave','imdbtitle']].tail(3))
#print(revenue_df.shape)
#print(revenue_df[['llave','title']].tail(3))

print(all_joined_df.shape)

Start with revenue data for 28,778 films.
Keep 2017 or earlier movies. Now have revenue data for 28,536 films.
Keep movies with for-reals titles. Now have revenue data for 28,502 films.
Before: 53 duplicate keys found...
After: 0 duplicate keys found!
Keep movies with box office releases, meaning revenue > 0. Now have revenue data for 15,259 films.
(19220, 47)


In [317]:
# Match requires a title field in from all three data sources
all_joined_df[['llave',
               'bechdel_title','bechdel_year',
               'imdbtitle','imdbyear',
               'title','year']][(~pd.isnull(all_joined_df['bechdel_title'])) & (~pd.isnull(all_joined_df['imdbtitle'])) & (~pd.isnull(all_joined_df['title']))].shape

# On the first try, there are 3,247 such rows. Scan a 1% random subset to make sure they contain true matches.
#all_joined_df[['llave',
#               'bechdel_title','bechdel_year',
#               'imdbtitle','imdbyear',
#               'title','year']][(~pd.isnull(all_joined_df['bechdel_title'])) & (~pd.isnull(all_joined_df['imdbtitle'])) & (~pd.isnull(all_joined_df['title']))].sample(frac=0.01)
# Yes, all look like true matches!
#all_joined_df

(3293, 7)

In [316]:
# Now, let's see what did not match
matcher = all_joined_df[['llave',
               'bechdel_title','bechdel_year',
               'imdbtitle','imdbyear',
               'title','year']][(pd.isnull(all_joined_df['bechdel_title'])) | (pd.isnull(all_joined_df['imdbtitle'])) | (pd.isnull(all_joined_df['title']))]               
# Earliest revenue matches from 1915. So no matches before 1915.
#print('{0:,.0f} rows have Bechdel titles in need of matching.'.format(len(matcher[~pd.isnull(matcher['bechdel_title'])])))

#matcher.groupby('year').size()
#print(matcher[['llave','bechdel_title']][matcher['bechdel_year'] == 1945].sort_values('bechdel_title'))
#print(matcher[['llave','title','year']][matcher['year'] == 1945].sort_values('title'))

matcher = matcher[(matcher['bechdel_year'] > 1945) | (pd.isnull(matcher['bechdel_year']))]

matcher = matcher[((matcher['bechdel_title'].str.lower().str.slice(0,1)) != 'a') | (pd.isnull(matcher['bechdel_title']))]

matcher['bechdel_title'] = matcher['bechdel_title'].str.slice(0,40)
print('{0:,.0f} rows have Bechdel titles in need of matching.'.format(len(matcher[~pd.isnull(matcher['bechdel_title'])])))
#print('\n')

print(matcher[['llave','bechdel_title','bechdel_year']]
      [matcher['bechdel_title'].str.lower().str.slice(0,2) == 'az'].sort_values('bechdel_title'))
print('\n')
print(matcher[['llave','title','year']].dropna()
      [matcher['title'].str.lower().dropna().str.slice(0,2) == 'az'].sort_values('title'))

#print(matcher[['llave','title']].dropna()[matcher['title'].dropna().str.contains('Awakening')])

#revenue_df.index = np.where((revenue_df.index == 'casabla42anca'), 'casabla43anca',revenue_df.index)
  



3,322 rows have Bechdel titles in need of matching.
Empty DataFrame
Columns: [llave, bechdel_title, bechdel_year]
Index: []


                 llave                title    year
imdb_id                                            
6264596  azulosc07egro  Azuloscurocasinegro  2007.0
6264596  azumi06zumi    Azumi                2006.0
6264596  azúcara96arga  Azúcar amarga        1996.0
